In [1]:
import pandas as pd

In [34]:
fn = 'PH 211 W17 (Baldwin 32032)-Reading Questions (Week 1 Friday)-responses.csv'

In [36]:
df.drop?

In [37]:
df = (pd.read_csv(fn)
        .drop(['Institution', 'Department'], axis=1)
        .rename(columns=lambda string: string.replace(' ', '_')))

In [38]:
df.columns

Index(['Last_name', 'First_name', 'ID_number', 'Email_address', 'State',
       'Started_on', 'Completed', 'Time_taken', 'Response_1', 'Response_2'],
      dtype='object')

In [83]:
def clean_resps(df, resp_col = 'Response_1'):
    cols = [col for col in df.columns if not col.startswith('Response')]
    def make_response(row):
        resp = dict(row[cols])
        resp['Response'] = row[resp_col]
        return resp
    resps = [make_response(row) for i, row in df.iterrows()]
    return resps

In [84]:
resps = clean_resps(df)

In [110]:
import re
from jinja2 import evalcontextfilter, Markup, escape

_paragraph_re = re.compile(r'(?:\r\n|\r|\n){2,}')

@evalcontextfilter
def nl2br(eval_ctx, value):
    result = u'\n\n'.join(u'<p>%s</p>' % p.replace('\n', Markup('<br>\n'))
                          for p in _paragraph_re.split(escape(value)))
    if eval_ctx.autoescape:
        result = Markup(result)
    return result

In [113]:
import jinja2

loader = jinja2.FileSystemLoader('.')
env = jinja2.Environment(loader=loader)
env.filters['nl2br'] = nl2br

In [114]:
template = env.get_template('template.html')

In [115]:
qs = [col for col in df.columns if col.startswith('Response')]

qs

['Response_1', 'Response_2']

In [116]:
for q in qs:
    resps = clean_resps(df, q)
    out = template.render(resps=resps, subject="Reading questions", headline=q)
    with open('%s.html' % q, 'w') as fl:
        fl.write(out)

In [117]:
!open Response_1.html